O objetivo do trabalho da M2.2 é trabalhar com esteganografia em imagens. Dada uma imagem, devem ser implementados os algoritmos para esconder uma mensagem e para ler uma mensagem escondida.

Essa mensagem deve ser escondida no último bit da cor vermelha (não precisa trabalhar com binários, somente com valores pares e ímpares) e deve ser retirada uma mensagem de lá.

Precisam ser criadas duas funções: uma que crie uma lista de inteiros (com valores de 0 ou 1) composta pelos últimos dígitos da cor vermelha da imagem; a outra função dever receber uma lista de inteiros como parâmetro e alterar o último bit da cor vermelha da imagem para cada elemento. Lembre-se que a lista não vai ser grande o suficiente para toda a imagem, então altere a imagem só até a mensagem terminar.

Não precisam se preocupar com a conversão da mensagem de texto para a lista de bits e nem o contrário, abaixo já existem essas duas funções prontas: `gerar_mensagem()` recebe uma string por parâmetro e gera uma lista de inteiros e `converter_mensagem()` recebe uma lista de inteiros e gera uma mensagem.

**O trabalho deve utilizar essas duas funções, para o trabalho feito sem elas será atribuída nota 0.**

In [2]:
import numpy as np
import cv2 as cv

In [45]:
def bitfield(n):
    return [int(digit) for digit in bin(n)[2:]]

def gerar_mensagem(mensagem):
    lista = []
    for m in mensagem:
        val = ord(m)
        bits = bitfield(val)

        if len(bits) < 8:
            for a in range(8-len(bits)):
                bits.insert(0,0)
        lista.append(bits)
    arr = np.array(lista)
    arr = arr.flatten()
    return arr


def converter_mensagem(saida):
    bits = np.array(saida)
    mensagem_out = ''
    bits = bits.reshape((int(len(saida)/8), 8))
    for b in bits:
        sum = 0
        for i in range(8):
            sum += b[i]*(2**(7-i))
        mensagem_out += chr(sum)
    return mensagem_out

# texto = "Outra mensagem muito legal escrita agora"
# arrayBits = gerar_mensagem(texto)
# print(texto)
# print(arrayBits)
# textoTraduzido = converter_mensagem(arrayBits)
# print(textoTraduzido)

In [46]:
"""
//TODO:
- Função que crie uma lista de inteiros (com valores de 0 ou 1) composta pelos últimos dígitos da cor vermelha da imagem - OK
- Função que dever receber uma lista de inteiros como parâmetro e alterar o último bit da cor vermelha da imagem para cada elemento. Lembre-se que a lista não vai ser grande o suficiente para toda a imagem, então altere a imagem só até a mensagem terminar - OK

// Info:
# [largura, altura, canal_de_cor]
# [:,:,2] === Canal Vermelho de Cor
# Evitar que a mensagem distorça a imagem
# Pegar o LSB (Least Significant Bit) da cor desejada (vermelha)
# Usar par e ímpar, ao invés de trabalhar direto com bits
"""

def hide_message_in_image(message_bits, img_array):    
    # O loop abaixo é usado para "limpar" a imagem, evitando que ao decodificar a msg fique um monte de letras sem sentido
    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]):
            if img_array[i, j, 2] % 2 != 0: # Validar se é ímpar
                img_array[i, j, 2] += - 1 # Deixa o valor ímpar em par, usado para setar o LSB durante a conversão de bits em ASCII. PS: alterar o valor direto pra distorçe a imagem e deixa a imagem modificadas

    counter = 0 # Controle do tamanho da mensagem
    for i in range(img_array.shape[0]):
        for j in range(img_array.shape[1]): 
            if message_bits[counter] == 1:
                img_array[i, j, 2] += 1 # Se for 1 na posição atual da message_bits, vai transformar o valores atual na imagem em ímpar
            counter += 1

            if counter >= len(message_bits): # Controle para evitar que a mensagem altere mais valores do que deve
                break
        if counter >= len(message_bits): # Como está em um nested loop necessário usar break em todos os loops
            break

    return img_array

def show_message_in_image(img_with_secret): 
    bits = []

    for i in range(img_with_secret.shape[0]):
        for j in range(img_with_secret.shape[1]): 
            if img_with_secret[i, j, 2] % 2 == 0: # Verifica se na imagem o valor atual é par e adiciona o bit 0, senão o bit 1
                bits.append(0) 
            else:
                bits.append(1) 

    return bits

def remove_msg_null_values(message):
    final_message = ""
    
    # Isso vai remover valores que não sejam a mensagem em si
    for char in message:
        if char != chr(0): # O número char 0 na table ASCII é NULL
            final_message += char

    return final_message

In [47]:
# Execução das Funções
img = cv.imread("./lenna.png")
message = "HEEEEEEEEEEEEEEEEEEEEEEEEEELLLLLPPPPPPPPP"
message_bits = gerar_mensagem(message)
          
print("Encoding...")
new_img = hide_message_in_image(message_bits, img)
cv.imwrite("saved_img.png", new_img) # Usando imwrite() por causa de um bug com o Linux e a opencv imshow()
print("Encoded!!!")

print("")

print("Decoding...")
secret_msg = show_message_in_image(new_img)
print("Decoded!!!")
print("The message is: ")
msg_from_img = converter_mensagem(secret_msg)
print(remove_msg_null_values(msg_from_img))

Encoding...
Encoded!!!

Decoding...
Decoded!!!
The message is: 
HEEEEEEEEEEEEEEEEEEEEEEEEEELLLLLPPPPPPPPP
